In [1]:
#
# Import needed libraries
#

from nltk.classify        import NaiveBayesClassifier
from nltk.corpus          import subjectivity
from nltk.sentiment       import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util  import *
from nltk                 import tokenize

In [2]:
# take information
n_instances = 100
subj_docs   = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs    = [(sent,  'obj') for sent in subjectivity.sents(categories= 'obj')[:n_instances]]

In [3]:
# split into train and test the objective and subjective documents
train_subj_docs = subj_docs[:80]
train_obj_docs  =  obj_docs[:80]
test_subj_docs  = subj_docs[80:100]
test_obj_docs   =  obj_docs[80:100]

In [4]:
# get together train and test 
# Remember, first subjective docs, then objective docs
training_docs   = train_subj_docs + train_obj_docs
testing_docs    =  test_subj_docs +  test_obj_docs

In [5]:
# IMPORTANT NOTE : https://www.nltk.org/_modules/nltk/sentiment/sentiment_analyzer.html#SentimentAnalyzer
# Analyzer is a tool taking a classifier and applying feature extraction. 
# it takes a classifier as an argument
# for more, check share link
sentim_analyzer = SentimentAnalyzer()

# negation marks all the words after a negation word was introduced
# takes care of change of meaning
all_words_neg   = sentim_analyzer.all_words(\
            [mark_negation(doc) for doc in training_docs])

In [6]:
# extract all unigrams - the words with a certain repeating frequency
unigram_feats   = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
#print(unigram_feats)

# create the functionality of extracting unigram feats to the sentim_analyzer 
# used in apply_features
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)


In [7]:
# get together the sets
training_set = sentim_analyzer.apply_features(training_docs)
test_set     = sentim_analyzer.apply_features( testing_docs)

In [8]:
# create the classifier
# https://www.nltk.org/_modules/nltk/classify/naivebayes.html
trainer      = NaiveBayesClassifier.train
classifier   = sentim_analyzer.train(trainer, training_set)

# Training classifier
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


In [9]:
print("\n\n<========== Training set ==========>")
print(training_set)

print("\n\n<========== Testing set ==========>")
print(test_set)
print("\n\n")



<========== Training set ==========>
[({'contains(.)': True, 'contains(the)': False, 'contains(,)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': False, 'contains(to)': False, 'contains(is)': True, 'contains(in)': False, 'contains(with)': False, 'contains(it)': False, 'contains(that)': False, 'contains(his)': False, 'contains(on)': False, 'contains(for)': False, 'contains(an)': False, 'contains(who)': False, 'contains(by)': False, 'contains(he)': False, 'contains(from)': False, 'contains(her)': False, 'contains(")': False, 'contains(film)': False, 'contains(as)': False, 'contains(this)': False, 'contains(movie)': False, 'contains(their)': False, 'contains(but)': False, 'contains(one)': True, 'contains(at)': False, 'contains(about)': True, 'contains(the_NEG)': False, 'contains(a_NEG)': False, 'contains(to_NEG)': False, 'contains(are)': False, "contains(there's)": False, 'contains(()': False, 'contains(story)': False, 'contains(when)': False, 'contains(so)': False, 

In [10]:
# examples
sentences = [
"VADER is smart, handsome, and funny.", # positive sentence example
"VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
"VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
"VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
"VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
"VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
"The book was good.",         # positive sentence
"The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
"The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
"A really bad, horrible book.",       # negative sentence with booster words
"At least it isn't a horrible book.", # negated negative sentence with contraction
":) and :D",     # emoticons handled
"",              # an empty string is correctly handled
"Today sux",     #  negative slang handled
"Today sux!",    #  negative slang with punctuation emphasis handled
"Today SUX!",    #  negative slang with capitalization emphasis
"Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
]

In [11]:
paragraph = "It was one of the worst movies I've seen, despite good reviews. \
Unbelievably bad acting!! Poor direction. VERY poor production. \
The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

lines_list = tokenize.sent_tokenize(paragraph)
sentences.extend(lines_list)
print(lines_list)

["It was one of the worst movies I've seen, despite good reviews.", 'Unbelievably bad acting!!', 'Poor direction.', 'VERY poor production.', 'The movie was bad.', 'Very bad movie.', 'VERY bad movie.', 'VERY BAD movie.', 'VERY BAD movie!']


In [12]:
tricky_sentences = [
"Most automated sentiment analysis tools are shit.",
"VADER sentiment analysis is the shit.",
"Sentiment analysis has never been good.",
"Sentiment analysis with VADER has never been this good.",
"Warren Beatty has never been so entertaining.",
"I won't say that the movie is astounding and I wouldn't claim that \
the movie is too banal either.",
"I like to hate Michael Bay films, but I couldn't fault this one",
"It's one thing to watch an Uwe Boll film, but another thing entirely \
to pay for it",
"The movie was too good",
"This movie was actually neither that funny, nor super witty.",
"This movie doesn't care about cleverness, wit or any other kind of \
intelligent humor.",
"Those who find ugly meanings in beautiful things are corrupt without \
being charming.",
"There are slow and repetitive parts, BUT it has just enough spice to \
keep it interesting.",
"The script is not fantastic, but the acting is decent and the cinematography \
is EXCELLENT!",
"Roger Dodger is one of the most compelling variations on this theme.",
"Roger Dodger is one of the least compelling variations on this theme.",
"Roger Dodger is at least compelling as a variation on the theme.",
"they fall in love with the product",
"but then it breaks",
"usually around the time the 90 day warranty expires",
"the twin towers collapsed today",
"However, Mr. Carter solemnly argues, his client carried out the kidnapping \
under orders and in the ''least offensive way possible.''"
]
sentences.extend(tricky_sentences)

In [13]:
# Analyze
sid = SentimentIntensityAnalyzer()
for sentence in sentences:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
        print()

VADER is smart, handsome, and funny.
compound: 0.8316, 
neg: 0.0, 
neu: 0.254, 
pos: 0.746, 
VADER is smart, handsome, and funny!
compound: 0.8439, 
neg: 0.0, 
neu: 0.248, 
pos: 0.752, 
VADER is very smart, handsome, and funny.
compound: 0.8545, 
neg: 0.0, 
neu: 0.299, 
pos: 0.701, 
VADER is VERY SMART, handsome, and FUNNY.
compound: 0.9227, 
neg: 0.0, 
neu: 0.246, 
pos: 0.754, 
VADER is VERY SMART, handsome, and FUNNY!!!
compound: 0.9342, 
neg: 0.0, 
neu: 0.233, 
pos: 0.767, 
VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
compound: 0.9469, 
neg: 0.0, 
neu: 0.294, 
pos: 0.706, 
The book was good.
compound: 0.4404, 
neg: 0.0, 
neu: 0.508, 
pos: 0.492, 
The book was kind of good.
compound: 0.3832, 
neg: 0.0, 
neu: 0.657, 
pos: 0.343, 
The plot was good, but the characters are uncompelling and the dialog is not great.
compound: -0.7042, 
neg: 0.327, 
neu: 0.579, 
pos: 0.094, 
A really bad, horrible book.
compound: -0.8211, 
neg: 0.791, 
neu: 0.209, 
pos: 0.0, 
At least it i